In [2]:
import json
import os

def mot_to_json(input_file):
    # Create a dictionary to store the JSON structure
    result = {}

    with open(input_file, 'r') as f:
        for line in f:
            # Split and parse the line
            frame, obj_id, bb_left, bb_top, bb_width, bb_height, _, _, _, _ = line.strip().split(',')
            frame = str(int(frame)).zfill(4)  # Format frame number as 4-digit string
            obj_id = str(int(obj_id))

            # Create bounding box in the [left, top, right, bottom] format
            bbox = [
                float(bb_left),
                float(bb_top),
                float(bb_left) + float(bb_width),
                float(bb_top) + float(bb_height)
            ]

            # Insert data into the result dictionary
            if frame not in result:
                result[frame] = {"person": {}, "car": {}}
            result[frame]["person"][obj_id] = bbox

    # Write the result to a JSON file
    output_file = "bbox_" + input_file.split('/')[-1].replace('.txt', '.json')
    with open(output_file, 'w') as jf:
        json.dump(result, jf, indent=4)

    print(f"Saved result to {output_file}")


# Example usage
# input_file = 'txt_archive/b1c9c847-3bda4659.txt'
# mot_to_json(input_file)


In [3]:

def process_txt_archive(txt_folder):
    # Iterate over all files in the txt_archive folder
    for filename in os.listdir(txt_folder):
        if filename.endswith(".txt"):
            input_file = os.path.join(txt_folder, filename)

            mot_to_json(input_file)

process_txt_archive('txt_archive')

Saved result to bbox_txt_archive\video_0001.json
Saved result to bbox_txt_archive\video_0003.json
Saved result to bbox_txt_archive\video_0055.json
Saved result to bbox_txt_archive\video_0056.json
Saved result to bbox_txt_archive\video_0057.json
Saved result to bbox_txt_archive\video_0194.json
Saved result to bbox_txt_archive\video_0310.json
Saved result to bbox_txt_archive\video_0313.json
Saved result to bbox_txt_archive\video_0333.json
Saved result to bbox_txt_archive\video_0343.json
